In [1]:
%matplotlib widget
%matplotlib inline
import sys
from sys import platform
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.ticker import ScalarFormatter
import numpy as np
import seaborn as sns
import scipy
from IPython.display import display, Math
from tqdm import tqdm

from astropy.io import fits, ascii
from astropy.table import Table
from astropy.modeling import functional_models, fitting
from astropy.timeseries import LombScargle

import stingray.events as ev
import stingray.lightcurve as lc
from stingray import io
import stingray.powerspectrum as powspec 
import stingray.crossspectrum as crossspec
import stingray.gti as sting_gti
import stingray.pulse.pulsar as plsr
import stingray.pulse.accelsearch as accelsearch
from stingray import stats


sns.set_context('talk')
# sns.set_style("whitegrid")
sns.set_palette("colorblind")

sys.path.insert(1, '/Users/sean/scripts/helpers')

from stingray_plus import *


/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/crossspectrum.py:21: UserWarning: Using standard scipy fft
  warnings.warn("Using standard scipy fft")


## Setup

In [2]:
OBSID = '30002032005'

if platform=='linux' or platform=='linux2':
    print('Working on SRL server')
    root_dir = '/disk/lif2/spike/IC_342/'
elif platform=='darwin':
    print('Working on Macbook')
    root_dir = '/Volumes/Samsung_1TB/AstroData/IC_342/'



Working on Macbook


In [3]:
plt.ion()
buff = False
lc_buff_size = 60
bkg_bin = 32
print(OBSID)

### Setup
products_dir = root_dir + OBSID + '_products/'
plot_dir = root_dir + 'figures/'
plt.ion()

events = extract_events(products_dir + 'nu' + OBSID + 'A01_cl_barycorr.evt', \
            products_dir + 'nu' + OBSID + 'B01_cl_barycorr.evt')

print(events[0].gti[0][0])
print(events[1].gti[0][0])

curve_A = bkg_subtract(nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'A01_sr.lc', buff = buff, buffersize = lc_buff_size), \
                       nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'A01_bk.lc', buff = buff, buffersize = lc_buff_size), bkg_bin=bkg_bin)
curve_B = bkg_subtract(nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'B01_sr.lc', buff = buff, buffersize = lc_buff_size),\
                       nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'B01_bk.lc', buff = buff, buffersize = lc_buff_size), bkg_bin=bkg_bin)
curve_total = sum_lc(curve_A, curve_B)
# curve_10s = curve_total.rebin(dt_new=10)

centroid_A = curve_A.centroid
centroid_B = curve_B.centroid
extraction_radius = curve_A.radius
#     print(centroid_A)
#     print(centroid_B)
#     print(extraction_radius)

t_start = np.min(curve_total.time)

## Plot the lightcurve
plt.figure(figsize = (9,6))
plt.errorbar(curve_total.time-t_start, curve_total.countrate, xerr=curve_total.dt/2., yerr=curve_total.countrate_err, fmt='none', lw = 0.5)
plt.xlabel('Time (s)')
plt.ylabel('NuSTAR count rate')
plt.tight_layout()
plt.savefig(plot_dir + 'bkg_sub_lightcurve.pdf')
plt.close()



30002032005
82801578.82400896
82801578.82400896


In [4]:
bin_time = 1/64
split_time = 64
keV_min = 6
keV_max = 30
PI_min = int(eV_to_PI(keV_min * 1000))
PI_max = int(eV_to_PI(keV_max * 1000))

### Create CPDS
curve_A = events[0].to_lc(dt = bin_time, pi_low=PI_min, pi_high=PI_max, centroid = centroid_A, radius = extraction_radius)
curve_B = events[1].to_lc(dt = bin_time, pi_low=PI_min, pi_high=PI_max, centroid = centroid_B, radius = extraction_radius)

averaged_cross = crossspec.AveragedCrossspectrum(curve_A, curve_B, segment_size=split_time, norm = 'frac')

### Plot CPDS
f_res = 0.01
averaged_cross_log = averaged_cross.rebin_log(f=f_res)
averaged_cross_log_err = averaged_cross.df*np.power(1.+f_res, range(len(averaged_cross_log.freq)))/2.

plt.figure(figsize=(9,6))
plt.errorbar(averaged_cross_log.freq, averaged_cross_log.power*averaged_cross_log.freq, xerr=averaged_cross_log_err, yerr=averaged_cross_log.power_err*averaged_cross_log.freq, fmt='none', color='black', lw=0.5)
plt.step(np.concatenate([averaged_cross_log.freq-averaged_cross_log_err, [averaged_cross_log.freq[-1]+averaged_cross_log_err[-1]]]), \
         np.concatenate([averaged_cross_log.power*averaged_cross_log.freq, [(averaged_cross_log.power*averaged_cross_log.freq)[-1]]]), where='post', color='black', lw=0.5)
plt.xscale('log')
plt.ylim(bottom=1e-3)
plt.yscale('log')
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'$\mathrm{\nu P_{\nu}\ (rms/mean)^{2}}$')
plt.tight_layout()
# plt.show()
plt.savefig(plot_dir + 'CPDS_nuPnu_' + str(int(split_time)) + 's_' + str(keV_min)+'_'+str(keV_max) + 'keV.pdf')
plt.close()

plt.figure(figsize=(9,6))
plt.errorbar(averaged_cross_log.freq, averaged_cross_log.power, xerr=averaged_cross_log_err, yerr=averaged_cross_log.power_err, fmt='none', color='black', lw=0.5)
plt.step(np.concatenate([averaged_cross_log.freq-averaged_cross_log_err, [averaged_cross_log.freq[-1]+averaged_cross_log_err[-1]]]), \
         np.concatenate([averaged_cross_log.power, [averaged_cross_log.power[-1]]]), where='post', color='black', lw=0.5)
plt.xscale('log')
plt.ylim(bottom=1e-3)
plt.yscale('log')
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'$\mathrm{Power\ (rms/mean)^{2}/Hz}$')
plt.tight_layout()
# plt.show()
plt.savefig(plot_dir + 'CPDS_Pnu_' + str(int(split_time)) + 's_' + str(keV_min)+'_'+str(keV_max) + 'keV.pdf')
plt.close()

### Fit one zero-centered Lorentzian
# popt, pcov, chisq0, chisq = model_continuum_noise_zero_center(averaged_cross, plot=True, plot_dir=plot_dir, f_res=f_res)

# del_chisq = chisq - chisq0
# print(del_chisq/chisq0)

/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/utils.py:119: UserWarning: SIMON says: Errorbars on cross spectra are not thoroughly tested. Please report any inconsistencies.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)
29it [00:00, 58.35it/s]/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/crossspectrum.py:1178: UserWarning: No counts in interval 82805380.98807146--82805444.97244646s
  warnings.warn(
/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/crossspectrum.py:1178: UserWarning: No counts in interval 82805700.98807146--82805764.97244646s
  warnings.warn(
58it [00:00, 109.56it/s]/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/crossspectrum.py:1178: UserWarning: No counts in interval 82808138.37869646--82808202.36307146s
  warnings.warn(
/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/crossspectrum.py:1178: UserWarning: No counts in interval 82808266.37869646--82808330.36307146s
  warnings.war

In [6]:
bin_time = 1/64
split_time = 64
keV_min = 6
keV_max = 30
PI_min = int(eV_to_PI(keV_min * 1000))
PI_max = int(eV_to_PI(keV_max * 1000))

for i in range(6):
    
    events = extract_events(products_dir + 'nu' + OBSID + 'A01_cl_bc_src_epoch'+(str(i+1))+'.evt', \
            products_dir + 'nu' + OBSID + 'B01_cl_bc_src_epoch'+(str(i+1))+'.evt')

    ### Create CPDS
    curve_A = events[0].to_lc(dt = bin_time, pi_low=PI_min, pi_high=PI_max)
    curve_B = events[1].to_lc(dt = bin_time, pi_low=PI_min, pi_high=PI_max)

    averaged_cross = crossspec.AveragedCrossspectrum(curve_A, curve_B, segment_size=split_time, norm = 'frac')

    ### Plot CPDS
    f_res = 0.01
    averaged_cross_log = averaged_cross.rebin_log(f=f_res)
    averaged_cross_log_err = averaged_cross.df*np.power(1.+f_res, range(len(averaged_cross_log.freq)))/2.

    plt.figure(figsize=(9,6))
    plt.errorbar(averaged_cross_log.freq, averaged_cross_log.power*averaged_cross_log.freq, xerr=averaged_cross_log_err, yerr=averaged_cross_log.power_err*averaged_cross_log.freq, fmt='none', color='black', lw=0.5)
    plt.step(np.concatenate([averaged_cross_log.freq-averaged_cross_log_err, [averaged_cross_log.freq[-1]+averaged_cross_log_err[-1]]]), \
             np.concatenate([averaged_cross_log.power*averaged_cross_log.freq, [(averaged_cross_log.power*averaged_cross_log.freq)[-1]]]), where='post', color='black', lw=0.5)
    plt.xscale('log')
    plt.ylim(bottom=1e-3)
    plt.yscale('log')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel(r'$\mathrm{\nu P_{\nu}\ (rms/mean)^{2}}$')
    plt.tight_layout()
    # plt.show()
    plt.savefig(plot_dir + 'CPDS_nuPnu_' + str(int(split_time)) + 's_' + str(keV_min)+'_'+str(keV_max) + 'keV_epoch' + str(i+1)+'.pdf')
    plt.close()

    plt.figure(figsize=(9,6))
    plt.errorbar(averaged_cross_log.freq, averaged_cross_log.power, xerr=averaged_cross_log_err, yerr=averaged_cross_log.power_err, fmt='none', color='black', lw=0.5)
    plt.step(np.concatenate([averaged_cross_log.freq-averaged_cross_log_err, [averaged_cross_log.freq[-1]+averaged_cross_log_err[-1]]]), \
             np.concatenate([averaged_cross_log.power, [averaged_cross_log.power[-1]]]), where='post', color='black', lw=0.5)
    plt.xscale('log')
    plt.ylim(bottom=1e-3)
    plt.yscale('log')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel(r'$\mathrm{Power\ (rms/mean)^{2}/Hz}$')
    plt.tight_layout()
    # plt.show()
    plt.savefig(plot_dir + 'CPDS_Pnu_' + str(int(split_time)) + 's_' + str(keV_min)+'_'+str(keV_max) + 'keV_epoch' + str(i+1)+'.pdf')
    plt.close()


/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/utils.py:119: UserWarning: SIMON says: Errorbars on cross spectra are not thoroughly tested. Please report any inconsistencies.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)
32it [00:00, 113.84it/s]/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/crossspectrum.py:1178: UserWarning: No counts in interval 82805380.98807146--82805444.97244646s
  warnings.warn(
/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/crossspectrum.py:1178: UserWarning: No counts in interval 82805700.98807146--82805764.97244646s
  warnings.warn(
53it [00:00, 147.25it/s]/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/crossspectrum.py:1178: UserWarning: No counts in interval 82808138.37869646--82808202.36307146s
  warnings.warn(
/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/crossspectrum.py:1178: UserWarning: No counts in interval 82808266.37869646--82808330.36307146s
  warnings.wa

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Samsung_1TB/AstroData/IC_342/30002032005_products/nu30002032005A01_cl_bc_src_epoch6.evt'

In [6]:
# for i, OBSID in enumerate(OBSID_list):
#     print(OBSID)
#     for j in range(len(noise_params[i])):
#         print(noise_params[i][j])
#         print(np.sqrt(np.diag(noise_error[i][j])))

for i, OBSID in enumerate(OBSID_list):
    print(OBSID)
#     print(noise_params[i])
#     print(np.sqrt(np.diag(noise_error[i])))

    gamma = noise_params[i][0]
    A = noise_params[i][1]

    gamma_err = np.sqrt(np.diag(noise_error[i]))[0]
    A_err = np.sqrt(np.diag(noise_error[i]))[1]

    rms = np.sqrt(A/2)
    rms_err = A_err/(4*np.sqrt(A/2))
    
    print(gamma)
    print(gamma_err)
    print(rms*100)
    print(rms_err*100)

80502324002
7.4231098851414385
3.989131539035984
9.777979792600545
1.8574690318325409
80502324004
1.968161680871277
6.430980406486496
2.9271719420746707
3.37943981470223
80502324006
2.1087816949081897
2.4262763589956413
8.632848109553661
3.509628669386416
80502324008
0.6423837346964079
0.07488765569478932
26.592674709067342
1.093924090704803
80502324010
0.10319234754084575
0.00793209949986902
30.358032963860943
0.8147788680461053
80502324012
0.06384622043039354
0.011154124406335264
30.501233523962934
1.8453922864861643
80502324014
0.02844103128730107
0.008165643187988866
30.003571091919273
2.896020209663599
80502324016
1.3034185395536202
0.663880194147966
120.03300946078075
21.59464223128655


# RMS-Energy relation

In [44]:
keV_list = np.array([[3.0, 5.0], [5.0, 8.0], [8.0, 15.0], [15.0, 50.0]])
PI_list = eV_to_PI(keV_list*1000)
rms_list = []
rms_err_list = []
uplim_list = []

for x in PI_list:
    print(x)
    rms, rms_err, uplim = get_rms(events, [centroid_A, centroid_B], extraction_radius, PI_min=x[0], PI_max=x[1], nu_min=1e-4, nu_max=1., split_time=split_time, bin_time = bin_time, plot = False)
    rms_list.append(rms*100)
    rms_err_list.append(rms_err*100)
    uplim_list.append(uplim)
    





[35. 85.]


/Users/sean/anaconda/lib/python3.6/site-packages/stingray/utils.py:118: UserWarning: SIMON says: Errorbars on cross spectra are not thoroughly tested. Please report any inconsistencies.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)
150it [00:08, 18.18it/s]
150it [00:07, 20.92it/s]
150it [00:07, 20.16it/s]


[ 85. 160.]


150it [00:09, 16.30it/s]
150it [00:07, 19.96it/s]
150it [00:08, 18.75it/s]


[160. 335.]


150it [00:08, 17.59it/s]
150it [00:07, 20.82it/s]
150it [00:07, 19.47it/s]


[ 335. 1210.]


150it [00:08, 16.97it/s]
150it [00:08, 18.33it/s]
150it [00:07, 19.89it/s]


In [45]:
print(uplim_list)
fig, ax = plt.subplots(figsize=(9,6))
ax.errorbar((keV_list.T[0]+keV_list.T[1])/2, rms_list, xerr = (keV_list.T[0]-keV_list.T[1])/2, yerr = rms_err_list, fmt='none', lw=1.0, uplims=uplim_list)
ax.set_xscale('log')
# plt.yscale('log')
ax.set_ylabel(r'$\mathrm{rms\ (\%)}$')
ax.set_xlabel('Energy (keV)')
ax.set_xticks([3, 4, 5, 6, 7, 8,9, 20, 30, 40, 50], minor=True)
ax.set_xticks([10], minor=False)
ax.set_xticklabels(['3','','','','','','','','','','50'], minor=True)
ax.set_xticklabels(['10'], minor=False)
plt.tight_layout()
plt.savefig(plot_dir + 'rms_E.pdf')
plt.close()

[False, False, False, False]
